In [193]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, SpatialDropout1D, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical, np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import pdb
from gensim.models import KeyedVectors
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot

In [3]:
en_model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

In [87]:
en_model.vocab

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{',': <gensim.models.keyedvectors.Vocab at 0x2e1852e8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x2e185048>,
 '.': <gensim.models.keyedvectors.Vocab at 0x2e185358>,
 'and': <gensim.models.keyedvectors.Vocab at 0x2e185080>,
 'of': <gensim.models.keyedvectors.Vocab at 0x2e1853c8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x2e1850f0>,
 'in': <gensim.models.keyedvectors.Vocab at 0x2e185438>,
 'a': <gensim.models.keyedvectors.Vocab at 0x2e185160>,
 '"': <gensim.models.keyedvectors.Vocab at 0x2e185470>,
 ':': <gensim.models.keyedvectors.Vocab at 0x2e185198>,
 ')': <gensim.models.keyedvectors.Vocab at 0x2e1854a8>,
 'that': <gensim.models.keyedvectors.Vocab at 0x2e1851d0>,
 '(': <gensim.models.keyedvectors.Vocab at 0x2e185518>,
 'is': <gensim.models.keyedvectors.Vocab at 0x2e185208>,
 'for': <gensim.models.keyedvectors.Vocab at 0x2e185588>,
 'on': <gensim.models.keyedvectors.Vocab at 0x2e185278>,
 '*': <gensim.models.keyedvectors.Vocab at 0x2e1855f8>,
 'with': <gensim.models.keyedvecto

In [35]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [209]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [208]:
def vec_word(li):
    total_vecs=[]
    for word in li:
        if word in en_model.vocab:
            vector = en_model[word]
            total_vecs.append(vector)
    return np.array(total_vecs)

In [6]:
len(vec_word('computer'))

300

In [9]:
isear=pd.read_csv('isear_1.csv', header=-1)
isear=isear.drop(2 , axis=1)
isear[0][isear[0]=='guit']='guilt'

In [216]:
X=isear[1]
X=X.apply(clean)
y=isear[0]

In [211]:
def word_splits(series):
    word_splits=series.str.split(' ')
    return word_splits

In [212]:
splits=word_splits(X)
numbers_series=splits.apply(vec_word)

In [213]:
splits

0       [day, feel, close, partner, friend, feel, peac...
1       [every, time, imagine, someone, love, could, c...
2       [obviously, unjustly, treated, possibility, el...
3       [think, short, time, live, relate, period, lif...
4       [gathering, found, involuntarily, sitting, nex...
5       [realized, directing, feeling, discontent, par...
6       [feel, guilty, realize, consider, material, th...
7          [girlfriend, taken, exam, went, parent, place]
8           [when, first, time, realized, meaning, death]
9         [car, overtaking, another, forced, drive, road]
10      [recently, thought, hard, work, take, study, o...
11                   [found, bristle, liver, paste, tube]
12      [tired, unmotivated, shouted, girlfriend, brou...
13      [think, study, enough, weekend, think, able, a...
14                        [pas, examination, think, well]
15      [one, arranged, meet, someone, person, arrives...
16         [one, unjustly, accused, something, one, done]
17      [one, 

In [204]:
len(X)

7516

In [214]:
numbers_series.iloc[1].shape

(11, 300)

<i>Using non-normalized vector numbers? Check</i>

In [217]:
X_1 = []
y_1 = []
for index in range(0, len(numbers_series)):
    doc=numbers_series.iloc[index]
    for i in range(3, len(doc)):
        X_1.append(doc[i-3:i])
        y_1.append(y.iloc[index])

In [219]:
encoder=LabelEncoder()
encoder.fit(y_1)
y_1=encoder.transform(y_1)
y=np_utils.to_categorical(y_1)
y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [231]:
X_train, X_test, y_train, y_test = train_test_split(X_1, y, test_size = 0.2, random_state = 0)

In [234]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [235]:
y_test.shape

(10692, 7)

In [236]:
y_train.shape

(42768, 7)

In [237]:
len(X_train)

42768

In [110]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [111]:
X_train.shape

(53460, 3, 300)

In [112]:
y_train.shape

(53460,)

In [128]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=50, return_sequences=True, input_shape=(None, 300...)`
  """


In [139]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'mean_squared_error')

In [141]:
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 18s 419us/step - loss: 0.0495 - acc: 0.7586
Epoch 2/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0490 - acc: 0.7635
Epoch 3/25
42768/42768 [==============================] - 15s 358us/step - loss: 0.0488 - acc: 0.7621
Epoch 4/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0488 - acc: 0.7621
Epoch 5/25
42768/42768 [==============================] - 15s 359us/step - loss: 0.0486 - acc: 0.7650
Epoch 6/25
42768/42768 [==============================] - 15s 360us/step - loss: 0.0487 - acc: 0.7630
Epoch 7/25
42768/42768 [==============================] - 15s 362us/step - loss: 0.0485 - acc: 0.7640
Epoch 8/25
42768/42768 [==============================] - 16s 370us/step - loss: 0.0481 - acc: 0.7680
Epoch 9/25
42768/42768 [==============================] - 16s 367us/step - loss: 0.0481 - acc: 0.7662
Epoch 10/25
42768/42768 [==============================] - 16s 384us/step - loss: 

In [142]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 1s 117us/step


[0.10446360716760003, 0.48110736999625886]

In [135]:
regressor.metrics_names

['loss']

<h3>Initial model with .48 test accuracy</h3>

In [137]:
model_json = regressor.to_json()
with open("emotions_regressor.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_regressor.h5")
print("Saved model to disk")

Saved model to disk


<h3>Second model</h3>

In [165]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [161]:
adam=Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [166]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [173]:
regressor.fit(X_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
42768/42768 [==============================] - 32s 748us/step - loss: 0.8835 - acc: 0.6842
Epoch 2/25
42768/42768 [==============================] - 32s 737us/step - loss: 0.8675 - acc: 0.6885
Epoch 3/25
42768/42768 [==============================] - 32s 743us/step - loss: 0.8545 - acc: 0.6947
Epoch 4/25
42768/42768 [==============================] - 31s 735us/step - loss: 0.8418 - acc: 0.7000
Epoch 5/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8274 - acc: 0.7037
Epoch 6/25
42768/42768 [==============================] - 31s 732us/step - loss: 0.8193 - acc: 0.7093
Epoch 7/25
42768/42768 [==============================] - 32s 750us/step - loss: 0.8059 - acc: 0.71211s - l
Epoch 8/25
42768/42768 [==============================] - 32s 747us/step - loss: 0.8002 - acc: 0.7122
Epoch 9/25
42768/42768 [==============================] - 32s 757us/step - loss: 0.7883 - acc: 0.7179
Epoch 10/25
42768/42768 [==============================] - 33s 765us/step - 

In [174]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[1.791913282456489, 0.48054620276842497]

In [175]:
model_json = regressor.to_json()
with open("emotions_regressor_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
regressor.save_weights("emotions_regressor_2.h5")
print("Saved model to disk")

Saved model to disk


<h3>Third model</h3>

In [177]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(Dropout(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(Dropout(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100))
regressor.add(Dropout(0.5))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [180]:
adam=Adam(lr=0.0007, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [181]:
# Compiling the RNN
regressor.compile(optimizer = adam, metrics=['accuracy'], loss = 'categorical_crossentropy')

In [196]:
hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 33s 783us/step - loss: 0.5579 - acc: 0.8134 - val_loss: 2.3434 - val_acc: 0.4560
Epoch 2/100
 3776/42768 [=>............................] - ETA: 28s - loss: 0.5140 - acc: 0.8284

KeyboardInterrupt: 

In [189]:
regressor.evaluate(X_test, y_test)

10692/10692 [==============================] - 2s 215us/step


[2.214059761745226, 0.4675458286569398]

In [ ]:
pyplot.plot(hist.history['loss'][500:])
pyplot.plot(hist.history['val_loss'][500:])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

<h3>Fourth model</h3>

In [226]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True, input_dim=300))
regressor.add(SpatialDropout1D(0.5))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 100, return_sequences = True))
regressor.add(SpatialDropout1D(0.5))

# Adding a LSTM layer
regressor.add(LSTM(units = 100))

# Adding the output layer
regressor.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=100, return_sequences=True, input_shape=(None, 300...)`
  """


In [227]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', metrics=['accuracy'], loss = 'categorical_crossentropy')

In [ ]:
X_train.shape

In [238]:
model_4_hist=regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_test, y_test), shuffle=False)

Train on 42768 samples, validate on 10692 samples
Epoch 1/100
42768/42768 [==============================] - 43s 1000us/step - loss: 1.8514 - acc: 0.2293 - val_loss: 1.7700 - val_acc: 0.2788
Epoch 2/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7607 - acc: 0.2812 - val_loss: 1.7198 - val_acc: 0.3098
Epoch 3/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.7203 - acc: 0.3059 - val_loss: 1.6828 - val_acc: 0.3389
Epoch 4/100
42768/42768 [==============================] - 34s 791us/step - loss: 1.6804 - acc: 0.3327 - val_loss: 1.6587 - val_acc: 0.3534
Epoch 5/100
42768/42768 [==============================] - 34s 795us/step - loss: 1.6542 - acc: 0.3474 - val_loss: 1.6329 - val_acc: 0.3690
Epoch 6/100
42768/42768 [==============================] - 34s 789us/step - loss: 1.6295 - acc: 0.3625 - val_loss: 1.6260 - val_acc: 0.3715
Epoch 7/100
42768/42768 [==============================] - 34s 792us/step - loss: 1.6074 - acc: 0.3700 - val_

<h3>Further options</h3>

I could 

1.Gather and combine many emotional labelled datasets to have larger training data

2.Create new training data synthetically

3.Create labelled data using word averages (auto-labelling) data

4.Use Indico to get around 10,000 more labelled training data rows

<strong>Indico's emotions</strong>:
Anger, Fear, Joy, Sadness, Surprise

<strong>ISEAR's emotions</strong>: Anger, Fear, Joy, Sadness, Disgust, Shame, Guilt
